In [1]:
%load_ext autoreload
%autoreload 2

from cmipper import functions_creche, utils, config, parallelised_download_and_process, main
import xarray as xa

In [2]:
download_config_dict = utils.read_yaml(config.download_config)
model_info_dict = utils.read_yaml(config.model_info)
limited_model_dict = functions_creche.limit_model_info_dict(model_info_dict, download_config_dict)
limited_model_dict

{'EC-Earth3P-HR': {'resolution': 0.25,
  'experiment_ids': ['hist-1950', 'highres-future'],
  'member_ids': ['r1i1p2f1', 'r3i1p2f1'],
  'data_nodes': ['esgf.ceda.ac.uk', 'esgf-data1.llnl.gov'],
  'frequency': 'mon',
  'variable_dict': {'mlotst': {'include': True,
    'table_id': 'Omon',
    'plevels': [None]},
   'tos': {'include': True, 'table_id': 'Omon', 'plevels': [None]}}}}

In [6]:
main.main()